In [1]:
import keras
import tensorflow as tf
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import cifar10 # Keras CIFAR10 dataset
#from tqdm import tqdm_notebook as tqdm # Jupyter notebook should use this
import vgg16 as vgg16 # modified from  https://www.cs.toronto.edu/~frossard/post/vgg16/
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # choose which GPU 

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
"""Load data"""
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

"""Convert to one-hot labels"""
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

"""Display"""
print('Train data shape: ', x_train.shape)
print('Train label shape: ', y_train.shape)
print('Test data shape: ', x_train.shape)
print('Test label shape: ', y_train.shape)

Train data shape:  (50000, 32, 32, 3)
Train label shape:  (50000, 10)
Test data shape:  (50000, 32, 32, 3)
Test label shape:  (50000, 10)


In [3]:
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
X_224 = tf.image.resize_bicubic(X, [224,224]) # Bicubic interpolation
#vgg16_modified = vgg16.vgg16(X)
VGG16_modified = vgg16.vgg16(X_224, use_bias_in_fc=False) # Give our resized CIFAR10 to VGG16



"""get fc1 from VGG16"""
fc1 = VGG16_modified.get_fc1()

DEEP_FEATURE_LAYER = VGG16_modified.get_fc1()

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
#vgg16_model = vgg16.vgg16_modified(X)
#parameters = vgg16_model.get_architecture()
#DEEP_FEATURE_LAYER = vgg16_model.get_deep_feature()

VGG16_architecture = VGG16_modified.get_architecture()
weights = np.load('vgg16_weights.npz')
keys = sorted(weights.keys())
for i, v in enumerate(keys):
    try:
        sess.run(VGG16_architecture[i].assign(weights[v]))
        print(str('loading: '), v, np.shape(weights[v]))
    except:
        break       
print('Imagenet pre-trained VGG16-weights loaded')




loading:  conv1_1_W (3, 3, 3, 64)
loading:  conv1_1_b (64,)
loading:  conv1_2_W (3, 3, 64, 64)
loading:  conv1_2_b (64,)
loading:  conv2_1_W (3, 3, 64, 128)
loading:  conv2_1_b (128,)
loading:  conv2_2_W (3, 3, 128, 128)
loading:  conv2_2_b (128,)
loading:  conv3_1_W (3, 3, 128, 256)
loading:  conv3_1_b (256,)
loading:  conv3_2_W (3, 3, 256, 256)
loading:  conv3_2_b (256,)
loading:  conv3_3_W (3, 3, 256, 256)
loading:  conv3_3_b (256,)
loading:  conv4_1_W (3, 3, 256, 512)
loading:  conv4_1_b (512,)
loading:  conv4_2_W (3, 3, 512, 512)
loading:  conv4_2_b (512,)
loading:  conv4_3_W (3, 3, 512, 512)
loading:  conv4_3_b (512,)
loading:  conv5_1_W (3, 3, 512, 512)
loading:  conv5_1_b (512,)
loading:  conv5_2_W (3, 3, 512, 512)
loading:  conv5_2_b (512,)
loading:  conv5_3_W (3, 3, 512, 512)
loading:  conv5_3_b (512,)
loading:  fc6_W (25088, 4096)
loading:  fc6_b (4096,)
loading:  fc7_W (4096, 4096)
loading:  fc7_b (4096,)
loading:  fc8_W (4096, 1000)
loading:  fc8_b (1000,)
Imagenet pre-tra

In [4]:
DEEP_FEATURE_LAYER.shape

TensorShape([Dimension(None), Dimension(4096)])

In [5]:
df_train_x = []
df_test_x = []
for x in x_train:
    df = sess.run(DEEP_FEATURE_LAYER, feed_dict={X: [x]})
    df_train_x.append(df[0])
for x in x_test:
    df = sess.run(DEEP_FEATURE_LAYER, feed_dict={X: [x]})
    df_test_x.append(df[0])
df_train_x = np.array(df_train_x)
df_test_x = np.array(df_test_x)

In [8]:
'''save numpy array to file'''
np.save(arr=df_train_x, file='train_x_deep_feature')
np.save(arr=y_train, file='train_y_labels')
np.save(arr=df_test_x, file='test_x_deep_feature')
np.save(arr=y_test, file='test_y_labels')

In [14]:
BATCH_SIZE = 50

df_train_x = []
df_test_x = []
EPOCHS = 1  # total number of training epochs
epoch = 0
total_steps = int(len(x_train)/BATCH_SIZE)
loss_value = 0
for step in range(total_steps):
    #get training batch
    if step*BATCH_SIZE + BATCH_SIZE < len(x_train):
        BATCH_X = x_train[step*BATCH_SIZE: step*BATCH_SIZE+BATCH_SIZE]
        BATCH_Y = y_train[step*BATCH_SIZE: step*BATCH_SIZE+BATCH_SIZE]
    else:
        BATCH_X = x_train[step*BATCH_SIZE:]
        BATCH_Y = y_train[step*BATCH_SIZE:]
    #train
    df = sess.run(DEEP_FEATURE_LAYER, feed_dict={X: BATCH_X})
    df_train_x = 
